# **Projeto Completo: Dominando SQL com Python, Pandas e PandaSQL**

***Objetivo:*** *Projeto end-to-end que simula o trabalho de um analista de dados. O foco é, portanto, na exploração, limpeza e consulta de um dataset real da Netflix, cobrindo desde os comandos SQL mais básicos até os mais avançados.*

## **Fase 1:** Preparação do Ambiente e Carga dos Dados

In [1]:
!pip install pandasql

import pandas as pd
from pandasql import sqldf
import numpy as np

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26773 sha256=d69276b1c571630fc84ccb7aff2b1ce25d3ac956a51ff3a9ff013e001f7e6f10
  Stored in directory: /root/.cache/pip/wheels/15/a1/e7/6f92f295b5272ae5c02365e6b8fa19cb93f16a537090a1cf27
Successfully built pandasql


obs.: criando o "motor" que permite executar as queries SQL;
-> função globals() permite que a função acesse os DF no escopo global dos notebooks.

In [2]:
def run_sql(query):
  return sqldf(query, globals())

Carregando o dataset...

In [3]:
url = 'https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-04-20/netflix_titles.csv'

df_netflix = pd.read_csv(url)

Visualizando:

In [4]:
df_netflix.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


## **Fase 2:** Análise Exploratória e Limpeza de Dados (Data Cleaning)

In [5]:
print('Informações iniciais do DataFrame: ')

df_netflix.info()

Informações iniciais do DataFrame: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7787 entries, 0 to 7786
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       7787 non-null   object
 1   type          7787 non-null   object
 2   title         7787 non-null   object
 3   director      5398 non-null   object
 4   cast          7069 non-null   object
 5   country       7280 non-null   object
 6   date_added    7777 non-null   object
 7   release_year  7787 non-null   int64 
 8   rating        7780 non-null   object
 9   duration      7787 non-null   object
 10  listed_in     7787 non-null   object
 11  description   7787 non-null   object
dtypes: int64(1), object(11)
memory usage: 730.2+ KB


Observações iniciais:
- 'date_added' deveria ser do tipo datetime;
- observando as entradas (7787), vemos que algumas colunas ('director', 'cast' e 'country') têm valores nulos;
- 'duration' tem formatos mistos, necessitando uma padronização.

In [6]:
# tratando os valores nulos:

df_netflix['director'].fillna('Desconhecido', inplace=True)
df_netflix['cast'].fillna('Desconhecido', inplace=True)
df_netflix['country'].fillna('Desconhecido', inplace=True)

# e também:

df_netflix.dropna(subset=['date_added', 'rating'], inplace=True)

/tmp/ipython-input-2348035760.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_netflix['director'].fillna('Desconhecido', inplace=True)
/tmp/ipython-input-2348035760.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tru

In [7]:
# padronizando a coluna duration através da criação de 2 novas colunas:

df_netflix['duration_int'] = df_netflix['duration'].apply(lambda x: int(x.split(' ')[0]) if 'min' in x else 0)
df_netflix['seasons'] = df_netflix['duration'].apply(lambda x: int(x.split(' ')[0]) if 'Season' in x else 0)

In [8]:
# convertendo a coluna data_added para o tipo datetime

df_netflix['date_added'] = pd.to_datetime(df_netflix['date_added'].str.strip())

Verificando a limpeza:

In [9]:
df_netflix.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7770 entries, 0 to 7786
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   show_id       7770 non-null   object        
 1   type          7770 non-null   object        
 2   title         7770 non-null   object        
 3   director      7770 non-null   object        
 4   cast          7770 non-null   object        
 5   country       7770 non-null   object        
 6   date_added    7770 non-null   datetime64[ns]
 7   release_year  7770 non-null   int64         
 8   rating        7770 non-null   object        
 9   duration      7770 non-null   object        
 10  listed_in     7770 non-null   object        
 11  description   7770 non-null   object        
 12  duration_int  7770 non-null   int64         
 13  seasons       7770 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(10)
memory usage: 910.5+ KB


In [10]:
print('Amostra dos dados limpos com novas colunas: ')
df_netflix[['title', 'duration', 'duration_int', 'seasons']].head()

Amostra dos dados limpos com novas colunas: 


,title,duration,duration_int,seasons
0,3%,4 Seasons,0,4
1,7:19,93 min,93,0
2,23:59,78 min,78,0
3,9,80 min,80,0
4,21,123 min,123,0


In [11]:
df_netflix.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,duration_int,seasons
0,s1,TV Show,3%,Desconhecido,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,2020-08-14,2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...,0,4
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,2016-12-23,2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...,93,0
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,2018-12-20,2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow...",78,0
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,2017-11-16,2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi...",80,0
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,2020-01-01,2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...,123,0


## **Fase 3:** Guia de SQL através da biblioteca *PandaSQL*

### 3.1. SELEÇÃO E FILTRAGEM SIMPLES (SELECT, FROM, WHERE, DISTINCT, LIMIT)

Selecionar o título e tipo dos 5 primeiros conteúdos que são 'TV Show'

In [12]:
query_simples = '''
SELECT
DISTINCT type,
title
FROM df_netflix
WHERE type = 'TV Show'
LIMIT 5;'''

run_sql(query_simples)

,type,title
0,TV Show,3%
1,TV Show,46
2,TV Show,1983
3,TV Show,1994
4,TV Show,Feb-09


SELECT title, type: Escolhe as colunas title e type.

DISTINCT type: Garante que veremos apenas os valores únicos da coluna type.

FROM df_netflix: Indica que os dados vêm do nosso DataFrame df_netflix.

WHERE type = 'TV Show': Filtra as linhas, mantendo apenas aquelas onde o tipo é 'TV Show'.

LIMIT 5: Restringe a saída a, no máximo, 5 linhas.

### 3.1.1. USANDO LIKE PARA FLEXIBILIZAR A BUSCA

In [13]:
query_simples_2 = '''
SELECT type, title
FROM df_netflix
WHERE type like '%TV S%w%'
LIMIT 5;'''

run_sql(query_simples_2)

,type,title
0,TV Show,3%
1,TV Show,46
2,TV Show,1983
3,TV Show,1994
4,TV Show,Feb-09


A condição like é mais flexível que o "=", logo, é útil para certos tipos de busca.

### 3.2. ORDENANDO OS RESULTADOS (ORDER BY)

Listar os filmes pelo ano de lançamento, dos mais novos para os mais antigos.

In [14]:
query_3_2 = '''
SELECT title, release_year
FROM df_netflix
WHERE type = 'Movie'
ORDER BY release_year DESC;'''

run_sql(query_3_2)

,title,release_year
0,Charming,2021
1,Chris Rock Total Blackout: The Tamborine Exten...,2021
2,"Crack: Cocaine, Corruption & Conspiracy",2021
3,Creating The Queen's Gambit,2021
4,Outside the Wire,2021
...,...,...
5367,Undercover: How to Operate Behind Enemy Lines,1943
5368,Why We Fight: The Battle of Russia,1943
5369,WWII: Report from the Aleutians,1943
5370,Prelude to War,1942


ORDER BY release_year DESC: Ordena o resultado final com base na coluna release_year. DESC significa descendente (do maior para o menor). O padrão é ASC (ascendente).

### 3.3. FILTROS COMPLEXOS COM OPERADORES LÓGICOS (AND, OR, IN, LIKE, BETWEEN)

Encontrar filmes de terror ('Horror Movies') ou ficção científica ('Sci-Fi & Fantasy') lançados entre 2018 e 2020, e cujo título contenha a palavra 'Dead'.

In [15]:
query_3_3 = '''
SELECT title, listed_in, release_year
FROM df_netflix
WHERE type = 'Movie'
AND (listed_in LIKE '%Horror%' OR listed_in LIKE '%Sci-Fi%')
AND release_year BETWEEN 2018 AND 2020
AND title LIKE '%Dead%';'''

run_sql(query_3_3)

,title,listed_in,release_year
0,Day of the Dead: Bloodline,"Horror Movies, Independent Movies",2018
1,Deadcon,Horror Movies,2019


AND: Todas as condições devem ser verdadeiras.

OR: Pelo menos uma das condições dentro dos parênteses deve ser verdadeira.

LIKE '%Dead%': Busca por um padrão. O % é um curinga que representa qualquer sequência de caracteres.

BETWEEN 2018 AND 2020: Seleciona valores dentro de um intervalo (inclusivo).

### 3.4. AGREGAÇÃO DE DADOS (GROUP BY, COUNT, AVG, MAX, MIN)

Para cada classificação indicativa ('rating'), contar o número de títulos, calcular a média de duração dos filmes e o ano do filme mais antigo e mais novo.

In [16]:
query_3_4 = '''
SELECT rating,
COUNT(show_id) AS total_titulos,
AVG(duration_int) AS media_duracao,
MIN(release_year) AS ano_mais_antigo,
MAX(release_year) AS ano_mais_novo
FROM df_netflix
WHERE type = 'Movie' and duration_int > 0
GROUP BY rating
ORDER BY total_titulos DESC;'''

run_sql(query_3_4)

,rating,total_titulos,media_duracao,ano_mais_antigo,ano_mais_novo
0,TV-MA,1845,95.386450,1945,2021
1,TV-14,1272,110.709119,1942,2021
2,R,663,106.466063,1962,2021
3,TV-PG,505,94.786139,1943,2021
4,PG-13,386,107.772021,1955,2020
5,PG,247,98.526316,1973,2020
6,TV-Y,117,46.538462,2003,2020
7,TV-G,111,79.927928,1954,2020
8,TV-Y7,95,61.684211,2001,2021
9,NR,79,95.443038,1958,2018


GROUP BY rating: Agrupa todas as linhas com o mesmo rating para que as funções de agregação atuem em cada grupo.

COUNT(show_id): Conta o número de linhas (títulos) em cada grupo.

AVG(duration_int): Calcula a média da duração para os filmes de cada grupo.

MIN() e MAX(): Encontram o menor e o maior valor, respectivamente.

### 3.5. FILTRANDO GRUPOS (HAVING)
(É como um WHERE, mas para os resultados do GROUP BY)

Mostrar apenas as classificações indicativas que têm mais de 100 filmes e cuja duração média seja superior a 110 minutos.

In [17]:
query_3_5 = '''
SELECT rating,
COUNT(show_id) AS total_titulos,
AVG(duration_int) AS media_duracao_filmes
FROM df_netflix
WHERE type = 'Movie' and duration_int > 0
GROUP BY rating
HAVING COUNT(show_id) > 100 and AVG(duration_int) > 110
ORDER BY media_duracao_filmes DESC;'''

run_sql(query_3_5)

,rating,total_titulos,media_duracao_filmes
0,TV-14,1272,110.709119


### 3.6. UNINDO CONJUNTOS DE DADOS (INNER JOIN, LEFT JOIN)



In [18]:
# Primeiramente, como os comandos servem para combinar dados de diferentes tabelas, precisamos criar um novo DataFrame artificial:

dados_avaliacoes = dados_avaliacoes = {
    'title': ['The Crown', 'Stranger Things', 'The Queen\'s Gambit', 'Black Mirror', 'Inception', 'The Irishman'],
    'user_rating': [9.5, 9.8, 9.9, 9.7, 10.0, 8.5]
}

df_avaliacoes = pd.DataFrame(dados_avaliacoes)

### 3.6.1. INNER JOIN - Retorna apenas os títulos que existem em AMBOS os DataFrames

In [19]:
query_inner_join = '''
SELECT n.title,
n.type,
a.user_rating
FROM df_netflix AS n
INNER JOIN df_avaliacoes AS a ON n.title = a.title;'''

run_sql(query_inner_join)

,title,type,user_rating
0,Black Mirror,TV Show,9.7
1,Inception,Movie,10.0
2,Stranger Things,TV Show,9.8
3,The Crown,TV Show,9.5
4,The Irishman,Movie,8.5
5,The Queen's Gambit,TV Show,9.9


### 3.6.2.  LEFT JOIN - Retorna TODOS os títulos do DataFrame da esquerda (df_avaliacoes) e combina com os correspondentes da direita (df_netflix).

In [20]:
query_left_join = '''
SELECT a.title,
a.user_rating,
n.type,
n.release_year
FROM df_avaliacoes AS a
LEFT JOIN df_netflix AS n ON a.title = n.title;'''

run_sql(query_left_join)

,title,user_rating,type,release_year
0,The Crown,9.5,TV Show,2020
1,Stranger Things,9.8,TV Show,2019
2,The Queen's Gambit,9.9,TV Show,2020
3,Black Mirror,9.7,TV Show,2019
4,Inception,10.0,Movie,2010
5,The Irishman,8.5,Movie,2019


INNER JOIN: Funciona como uma interseção. Só aparecem resultados se a chave de junção (title) existir em ambos os DataFrames.

LEFT JOIN: Prioriza o DataFrame da esquerda. Traz todos os seus registros e busca correspondentes no da direita. Se não encontrar, preenche as colunas do DataFrame da direita com NULL.

### 3.7. CONSULTAS ANINHADAS (SUBQUERIES)

(Uma query dentro de outra query. Útil para criar filtros dinâmicos)

Selecionar todos os filmes com duração maior que a duração média de todos os filmes da Netflix

In [21]:
query_subquery = '''
SELECT title, duration_int
FROM df_netflix
WHERE type = 'Movie'
AND duration_int > (SELECT AVG(duration_int) from df_netflix WHERE type = 'Movie')
ORDER BY duration_int DESC
LIMIT 10;'''

run_sql(query_subquery)

,title,duration_int
0,Black Mirror: Bandersnatch,312
1,The School of Mischief,253
2,No Longer kids,237
3,Lock Your Girls In,233
4,Raya and Sakina,230
5,Sangam,228
6,Lagaan,224
7,Jodhaa Akbar,214
8,Kabhi Khushi Kabhie Gham,209
9,The Irishman,209


A subquery (SELECT AVG(duration_int) FROM df_netflix WHERE type = 'Movie') é executada primeiro, retornando um único valor (a média da duração).

A query principal então usa esse valor para filtrar os filmes, mostrando apenas aqueles com duration_int maior que a média calculada.

### 3.8. LÓGICA CONDICIONAL (CASE WHEN)

(Cria colunas derivadas com base em regras, como um "se-então-senão")

Classificar os títulos em categorias de duração: 'Curto' (< 60 min), 'Médio' (60-120 min) e 'Longo' (> 120 min).

In [22]:
query_case_when = '''
SELECT title, duration_int,
CASE WHEN duration_int < 60 THEN 'Curto'
WHEN duration_int BETWEEN 60 AND 120 THEN 'Médio'
ELSE 'Longo'
END AS categoria_duracao
FROM df_netflix
WHERE type = 'Movie' AND duration_int > 0
LIMIT 10;'''

run_sql(query_case_when)

,title,duration_int,categoria_duracao
0,7:19,93,Médio
1,23:59,78,Médio
2,9,80,Médio
3,21,123,Longo
4,122,95,Médio
5,187,119,Médio
6,706,118,Médio
7,1920,143,Longo
8,1922,103,Médio
9,"2,215",89,Médio


CASE WHEN avalia as condições em ordem. A primeira condição verdadeira determina o valor.

ELSE captura todos os casos que não se encaixam nas condições anteriores.

END AS categoria_duracao finaliza a expressão CASE e nomeia a nova coluna.

### 3.9. FUNÇÕES DE JANELA (WINDOW FUNCTIONS)

(Funções de janela executam cálculos em um conjunto de linhas relacionadas à linha atual. É um conceito avançado e poderoso)

Para cada país, classificar (rankear) as séries de TV pelo número de temporadas

In [24]:
query_window = '''
SELECT title, country, seasons,
RANK() OVER (PARTITION BY country ORDER BY seasons DESC) as ranking_no_pais
FROM df_netflix
WHERE type = 'TV Show' AND seasons > 0 AND country IN ('United States', 'United Kingdom', 'Japan')
LIMIT 15;'''

run_sql(query_window)

,title,country,seasons,ranking_no_pais
0,Naruto,Japan,9,1
1,Saint Seiya,Japan,6,2
2,Terrace House: Opening New Doors,Japan,6,2
3,Bleach,Japan,5,4
4,Fullmetal Alchemist: Brotherhood,Japan,5,4
5,Hunter X Hunter (2011),Japan,4,6
6,ONE PIECE,Japan,4,6
7,The Seven Deadly Sins,Japan,4,6
8,Aggretsuko,Japan,3,9
9,BAKI,Japan,3,9


RANK() OVER (...): A função RANK() atribui um ranking.

PARTITION BY country: Define as "janelas" ou grupos. O cálculo do ranking reinicia para cada novo country.

ORDER BY seasons DESC: Dentro de cada janela (país), as linhas são ordenadas pelo número de temporadas para que o RANK() seja aplicado corretamente.

## **Fase 4:** Mini-Projeto de Análise

In [35]:
# 1. Quais são os 5 países com mais filmes na Netflix?
q_paises = """
SELECT country, COUNT(*) AS total_filmes
FROM df_netflix
WHERE type = 'Movie' AND country != 'Desconhecido'
GROUP BY country
ORDER BY total_filmes DESC
LIMIT 5;"""

run_sql(q_paises)

,country,total_filmes
0,United States,1847
1,India,852
2,United Kingdom,193
3,Canada,118
4,Spain,89


In [34]:
# 2. Qual é o diretor que mais aparece no dataset (excluindo 'Desconhecido')?
q_diretor = """
SELECT director, COUNT(*) AS total_titulos
FROM df_netflix
WHERE director != 'Desconhecido'
GROUP BY director
ORDER BY total_titulos DESC
LIMIT 1;"""

run_sql(q_diretor)

,director,total_titulos
0,"Raúl Campos, Jan Suter",18


In [33]:
# 3. Quais são as 5 séries mais recentes adicionadas?
q_series_recentes = """
SELECT title, date_added
FROM df_netflix
WHERE type = 'TV Show'
ORDER BY date_added DESC
LIMIT 5;"""

run_sql(q_series_recentes)

,title,date_added
0,Bling Empire,2021-01-15 00:00:00.000000
1,Carmen Sandiego,2021-01-15 00:00:00.000000
2,Disenchantment,2021-01-15 00:00:00.000000
3,Henry Danger,2021-01-15 00:00:00.000000
4,Kuroko's Basketball,2021-01-15 00:00:00.000000


In [38]:
# 4. Qual a distribuição de filmes por década?
q_decada = """
SELECT (release_year / 10) * 10 || 's' AS decada,
COUNT(*) AS total_filmes
FROM df_netflix
WHERE type = 'Movie'
GROUP BY decada
ORDER BY decada;"""

run_sql(q_decada)

,decada,total_filmes
0,1940,13
1,1950,11
2,1960,22
3,1970,63
4,1980,99
5,1990,194
6,2000,601
7,2010,3946
8,2020,423


In [39]:
# 5. Qual é a média de duração dos filmes?
q_duracao_media = """
SELECT AVG(duration_int) AS media_duracao_minutos
FROM df_netflix
WHERE type = 'Movie' AND duration_int > 0;"""

run_sql(q_duracao_media)

,media_duracao_minutos
0,99.330417
